In [ ]:
# !pip install nltk

In [ ]:
import pandas as pd, numpy as np, re, json, ast, nltk, os
from string import punctuation
from tqdm import tqdm
import transformers
import torch

# Delete

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = pd.read_csv('drive/MyDrive/sirius/reviews.csv')

### вставить путь к датафрейму с отзывами
df = pd.read_csv('reviews_sravni.csv')
def process(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\n', ' ')
    return text

df['text'] = df['text'].apply(process)
df['text'].values[0]

'Выражаю свою благодарность и признательность персональному менеджеру Ирине Р. Работали с ней в течении 1 года безупречно, съели ни один пуд настроек, отчетов, справок, выписок, открытие новых р/с, эквайрингов и прочее. Доступно объянит, быстро решит любой вопрос. Впрочем, как и всегда при обращении в банк, поддержка клиентов — одна из лучших. Наша организация является клиентом еще двух банков, так что есть с чем сравнить. Ирина — вы лучшая! Спасибо за работу! Желаю процветания и роста только ввысь!'

In [ ]:
reviews = df['text'].values.tolist()

In [ ]:
# path='drive/MyDrive/sirius/data_banki_merged.json'
# data_without_labels_path='data_no_labels.txt'
# with open(data_without_labels_path, 'w') as f:
#   for sent in pd.read_json(path).iloc[:, 1].apply(lambda x: x['review']).values:
#     f.write(sent+'\n')

In [ ]:
# with open('/content/data_no_labels.txt', 'r') as f:
#     reviews = f.readlines()

# reviews = [line[:-1] for line in reviews]
# reviews = reviews
# reviews[0]

# Параметры обработки

In [ ]:
drop_stop_words = False # Удалить ли стоп-слова
drop_punctuation = True # Удалить ли пунктуацию
do_stemming = False # Применить ли стемминг
do_lemmatization = True # Применить ли лемматизацию
correct_punctuation = False # Исправить ли пунктуацию

do_augmentation_bert = False # Использовать ли аугментацию с помощью BERT
do_augmentation_typos = False # Использовать ли аугментацию с помощью добавления опечаток

# Что использовать для получения эмбеддингов
embedding_type = "fasttext" # @param ["bert", "tf-idf", "word2wec", "fasttext"]

# Requirements

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [ ]:
if do_lemmatization:
    !pip install pymorphy2
    !pip install pymorphy2-dicts
    !pip install DAWG-Python

    import pymorphy2

    lemmatizer = pymorphy2.MorphAnalyzer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c55c7204582f67e74168b288dfc7bbde830db328da093035bb21875c27997a2f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.8 MB/s eta 0:00:00


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
if correct_punctuation:
    !pip install git+https://huggingface.co/kontur-ai/sbert_punc_case_ru
    from sbert_punc_case_ru import SbertPuncCase
    model_punct = SbertPuncCase().to(device)

# Processing

In [ ]:
def add_spaces(reviews):
  # add spaces if needed
  for iter in range(3):
    c, c_bad=len(reviews), 0
    for i, review in enumerate(reviews):

      pattern=r'(\w\w[а-яё])([А-Я]\w\w)'    #Добавляем пробелы в нужных местах (по типу 'большойВвыбор')
      if re.search(pattern, review):
        c_bad+=1
        review=re.sub(pattern, r'\1 \2', review)
        reviews[i] = review

    print(f'Итерация {iter}, всего отзывов {c}, в {c_bad} добавлены пробелы')
  return reviews

reviews = add_spaces(reviews)

Итерация 0, всего отзывов 10000, в 62 добавлены пробелы
Итерация 1, всего отзывов 10000, в 3 добавлены пробелы
Итерация 2, всего отзывов 10000, в 0 добавлены пробелы


In [ ]:
# точка, запятая - не трогать, если по бокам цифры; иначе пробелы добавить (если не повторяются, '...' тоже не трогаем)
# остальные - добавить пробелы по бокам, если не повторяются (по типу !!!)

def process_punctuation(reviews, drop_punctuation=False, correct_punctuation=False):
  # Обрабатываем знаки препинания

    for i, review in enumerate(reviews):

        if drop_punctuation or correct_punctuation:
            review = re.sub(rf'[{punctuation}]', ' ', review) # Удаляем всю пунктуацию
            continue


        review= re.sub(rf'(\W)', rf' \1 ', review) # Банк,плохой -> Банк , плохой
        review = re.sub(rf'\s+', rf' ', review) # Банк    ,    плохой -> Банк , плохой

        for punct in '.,':
            review = re.sub(rf'([\d\{punct}]) (\{punct}) ([\d\{punct}])', rf'\1\2\3', review) # 1 , 5 -> 1,5 или . . . -> ...

        for punct in punctuation:
            review = re.sub(rf'(\{punct}) (\{punct}) (\{punct})', rf'\1\2\3', review) # ! ! ! -> !!!

        reviews[i] = review

    print(reviews[0].split()[:10])
    return reviews

reviews = process_punctuation(reviews, drop_punctuation, correct_punctuation)

['Выражаю', 'свою', 'благодарность', 'и', 'признательность', 'персональному', 'менеджеру', 'Ирине', 'Р.', 'Работали']


In [ ]:
def process_stop_words(reviews, stop_words):
    # Удаляем стоп-слова

    for i, review in enumerate(reviews):
        review = [word for word in review.split() if word.lower() not in stop_words]
        review = ' '.join(review)
        reviews[i] = review

    print(reviews[0].split()[:10])
    return reviews

if drop_stop_words:
    reviews = process_stop_words(reviews, stop_words)

## Стемминг

In [ ]:
def stemming(reviews, stemmer):
    # Применяем стемминг

    for i, review in tqdm(enumerate(reviews)):
        review = [stemmer.stem(word) for word in review.split()]
        reviews[i] = ' '.join(review)

    print(reviews[0].split()[:10])
    return reviews

if do_stemming:
    reviews = stemming(reviews, stemmer)

## Лемматизация

In [ ]:
def lemmatization(reviews, lemmatizer):
    # Применяем лемматизацию

    for i, review in tqdm(enumerate(reviews)):
        review = [lemmatizer.parse(word)[0].normal_form for word in review.split()]
        reviews[i] = ' '.join(review)

    print(reviews[0].split()[:10])
    return reviews

if do_lemmatization:
    reviews = lemmatization(reviews, lemmatizer)

10000it [03:38, 45.71it/s]

['выражать', 'свой', 'благодарность', 'и', 'признательность', 'персональный', 'менеджер', 'ирина', 'р.', 'работать']


## Правка пунктуации

In [ ]:
def punctuation_correction(reviews, model_punct, device):
    # Правим пунтуацию
    model_punct.to(device)

    for i, review in tqdm(enumerate(reviews)):
        review = model_punct.punctuate(review)
        reviews[i] = review

    return reviews

if correct_punctuation:
    reviews = punctuation_correction(reviews, model_punct, device)

## Аугментация

In [ ]:
def softmax(z):
    exp_z = np.exp(z)
    softmax_probs = exp_z / np.sum(exp_z, axis=0)
    return softmax_probs

def augment_data(reviews, do_augmentation_typos, do_augmentation_bert, fill_mask, tokenizer):
    prob_bert = 0.15 # Сколько слов заменяем с помощью BERT
    prob_typos = 0.15 # Сколько слов заменяем с помощью опечаток

    reviews_augmented = []

    for i, review in tqdm(enumerate(reviews)):
        if len(tokenizer.encode(review)) > 512:
            is_too_long=True
        else:
            is_too_long=False
        flag_review_changed = 0

        words = review.split()

        for j, word in enumerate(words):
            if word.isalnum() == False or word.isdigit(): # Цифры и пунктуацию не трогаем
                continue

            if do_augmentation_typos and do_augmentation_bert:
                choice = np.random.choice(3, 1, p=[1-prob_typos-prob_bert, prob_bert, prob_typos])[0] # 0 - пропуск, 1 - BERT, 2 - опечатка
            elif do_augmentation_typos:
                choice = np.random.choice(3, 1, p=[1-prob_typos, prob_typos])[0]
                if choice == 1:
                    choice = 2
            elif do_augmentation_bert:
                choice = np.random.choice(3, 1, p=[1-prob_bert, prob_bert])[0]


            if choice == 1:
                if is_too_long : continue # Слишком длинная последовательность
                words[i] = '[MASK]'
                for result in fill_mask(' '.join(words)):
                    if result['token_str'] != word and result['score']>=0.1: # Берем, если вероятность выше 0.1
                        flag_review_changed = 1
                        words[i] = result['token_str'] # Заменяем слово на новое
                        break
                else:
                    words[i] = word # Возвращаем слово на место

            elif choice == 2 and word in all_words:
                flag_review_changed = 1
                subset = mistakes.loc[mistakes['CORRECT']==word]
                p = softmax(subset.WEIGHT.values) # Вероятности ошибок
                new_word = subset.iloc[np.random.choice(np.arange(len(subset)), 1, p=p)[0], 1]
                words[i] = new_word

        review = ' '.join(words)
        if not flag_review_changed:
            reviews_augmented.append('')
        else:
            reviews_augmented.append(review)

    return reviews_augmented


if do_augmentation_bert or do_augmentation_typos:

    from transformers import pipeline

    mistakes = pd.read_csv('https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/orfo_and_typos/orfo_and_typos.L1_5.csv', sep=';')
    all_words = set(mistakes['CORRECT'].unique())

    tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base", truncation=True, max_length=512)

    fill_mask = pipeline('fill-mask', 'ai-forever/ruBert-base', tokenizer=tokenizer, device=device)


    reviews_augmented = augment_data(reviews, do_augmentation_typos, do_augmentation_bert, fill_mask, tokenizer)



## Получение эмбеддингов

### TF-IDF

In [ ]:
if embedding_type == 'tf-idf':
    from sklearn.feature_extraction.text import TfidfVectorizer
    max_df = 100 # Максимальная встречаемость слова
    min_df = 10 # Минимальная встречаемость слова
    vectorizer = TfidfVectorizer(input='content', max_df=max_df, min_df=min_df)

    embeddings = vectorizer.fit_transform(reviews)
    embeddings = embeddings.toarray()
    print(embeddings.shape)

### Word2vec

In [ ]:
def get_sentence_embedding(sentence, model):
    words = sentence.split()
    word_embeddings=[]
    for word in words:
        try:
            word_embeddings.append(model.wv.word_vec(word))
        except KeyError:
            pass
    if word_embeddings:
        return sum(word_embeddings) / len(word_embeddings) # Усредняем эмбеддинги слов в отзыве
    else:
        return None


if embedding_type == 'word2wec':

    !pip install gensim

    from gensim.models import Word2Vec

    w2v_model = Word2Vec(
        min_count=10,
        window=2,
        vector_size=300,
        negative=10,
        alpha=0.03,
        min_alpha=0.0007,
        sample=6e-5,
        sg=1)

    data = pd.Series([review.split() for review in reviews])

    w2v_model.build_vocab(data)

    embeddings = []
    for review in reviews:
        embeddings.append(get_sentence_embedding(review, w2v_model))

    embeddings = np.array(embeddings)
    print(embeddings.shape)

### fastText

In [ ]:
def get_sentence_embedding(sentence, model):
    words = sentence.split()
    word_embeddings=[]
    for word in words:
        try:
            word_embeddings.append(model[word])
        except KeyError:
            pass
    if word_embeddings:
        return sum(word_embeddings) / len(word_embeddings) # Усредняем эмбеддинги слов в отзыве
    else:
        return None


In [ ]:
if embedding_type == 'fasttext':
    !pip install fasttext

    import fasttext
    from huggingface_hub import hf_hub_download

    model_path = hf_hub_download(repo_id="facebook/fasttext-ru-vectors", filename="model.bin")
    model = fasttext.load_model(model_path)

    embeddings = []
    for review in reviews:
        embeddings.append(get_sentence_embedding(review, model))

    embeddings = np.array(embeddings)
    print(embeddings.shape)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=b3ebae8b9f2c24bebdfcfae84a77e1c78643277e968a435a3925f579ddbfa50e
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

(10000, 300)


### BERT

In [ ]:
from transformers import AutoTokenizer, AutoModel


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


if embedding_type == 'bert':


    tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
    model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

    model.to(device)

    embeddings = []

    for review in tqdm(reviews):

        encoded_input = tokenizer(review, padding=False, truncation=True, max_length=512, return_tensors='pt').to(device)

        with torch.no_grad():
            model_output = model(**encoded_input)

        embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        embeddings.append(embedding[0])

    embeddings = torch.stack(embeddings).cpu().numpy()



# Сохраним эмбеддинги

In [ ]:
embeddings = pd.DataFrame(embeddings)
embeddings.columns = [str(col) for col in embeddings.columns]
# embeddings.to_parquet('drive/MyDrive/sirius/embeddings_reviews_word2vec_sravni.parquet')
embeddings.to_parquet('drive/MyDrive/sirius/embeddings_reviews_fasttext_sravni.parquet')